## 简介

`Agent` 也就是 代理，它的核心思想是利用一个语言模型来选择一系列要执行的动作。`LangChain` 的链将一系列的动作硬编码在代码中。而在 `Agent` 中，语言模型被用作推理引擎，来确定应该执行哪些动作以及以何种顺序执行。

这就涉及到几个关键组件：

- `Agent` 代理
- `Tool` 工具
- `Toolkit` 工具包
- `AgentExecutor` 代理执行器

接下来我们做逐一介绍。注，该极简入门系列将略过工具包的介绍，这部分内容将包含在进阶系列中。

## Agent

`Agent` 由一个语言模型和一个提示词驱动，决定下一步要采取什么措施的类。提示词可以包括以下内容：

- 代理的个性（用于使其以特定方式回应）
- 代理的背景（用于为其提供更多关于所要执行任务类型的上下文信息）
- 引导策略（用于激发更好的推理能力）

`LangChain` 提供了不同类型的代理：

- Zero-shot ReAct
  
  利用 ReAct 框架根据工具的描述来决定使用哪个工具，可以使用多个工具，但需要为每个工具提供描述信息。工具的选择单纯依靠工具的描述信息。关于 ReAct 框架的更多信息，请参考 [ReAct](https://arxiv.org/pdf/2205.00445.pdf)。
  
- Structured Input ReAct
  
  相较于单一字符串作为输入的代理，该类型的代理可以通过工具的参数schema创建结构化的动作输入。
  
- OpenAI Functions
  
  该类型的代理用来与OpenAI Function Call机制配合工作。
  
- Conversational
  
  这类代理专为对话场景设计，使用具有对话性的提示词，利用 ReAct 框架选择工具，并利用记忆功能来保存对话历史。
  
- Self ask with search
  
  这类代理利用工具查找问题的事实性答案。
  
- ReAct document store
  
  利用 ReAct 框架与文档存储进行交互，使用时需要提供 `Search` 工具和 `Lookup` 工具，分别用于搜索文档和在最近找到的文档中查找术语。
  
- Plan-and-execute agents
  
  代理规划要做的事情，然后执行子任务来达到目标。
  

这里我们多次提到 “工具”，也就是 `Tool`，接下来我们就介绍什么是 `Tool`。

## Tool

`Tool` 工具，是代理调用的功能，通常用来与外部世界交互，比如维基百科搜索，资料库访问等。`LangChain` 内置的工具列表，请参考 [Tools](https://python.langchain.com/docs/integrations/tools/)。

## Toolkit

通常，在达成特定目标时，需要使用一组工具。`LangChain` 提供了 `Toolkit` 工具包的概念，将多个工具组合在一起。

## AgentExecutor

代理执行器是代理的运行时。程序运行中，由它来调用代理并执行其选择的动作。

## 组件实例

### Tool

`LangChain` 提供了一系列工具，比如 `Search` 工具，`AWS` 工具，`Wikipedia` 工具等。这些工具都是 `BaseTool` 的子类。通过调用 `run` 函数，执行工具的功能。

我们以 `LangChain` 内置的工具 `DuckDuckGoSearchRun` 为例，来看看如何使用工具。

注，要使用DuckDuckGoSearchRun工具，需要安装以下python包:
`pip install duckduckgo-search`

通过工具类创建工具实例

该类提供了通过 [`DuckDuckGo`](https://duckduckgo.com/) 搜索引擎搜索的功能。

In [1]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search.run("Who is winner of FIFA worldcup 2018?")

"The 2018 FIFA World Cup was the 21st FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA.It took place in Russia from 14 June to 15 July 2018, after the country was awarded the hosting rights in 2010. It was the eleventh time the championships had been held in Europe, and the first time they were held in Eastern Europe. They reverted back to type to battle past Uruguay and Belgium in the following rounds, but the brilliance of Mbappe, Antoine Griezmann and Paul Pogba shone out in their 4-2 final win over Croatia,... FIFA World Cup Qatar 2022™. World Cup memories of a French superfan. 6 May 2020. 2018 FIFA World Cup Russia™. #WorldCupAtHome: Electric Mbappe helps France win seven-goal thriller. 9 Apr 2020 ... World Cup 2018: The Winners and Losers. Luka Modric and Kylian Mbappe during the World Cup final. Jewel Samad/Agence France-Presse — Getty Images. France won the World Cup, but that is far from ... Pele and Brazil won their third title 

通过辅助函数 `load_tools` 加载

`LangChain` 提供了函数 `load_tools` 基于工具名称加载工具。

先来看看DuckDuckGoSearchRun类的定义：

In [6]:
from langchain.tools import BaseTool


class DuckDuckGoSearchRun(BaseTool):
    """Tool that adds the capability to query the DuckDuckGo search API."""

    name = "duckduckgo_search"
    description = (
        "A wrapper around DuckDuckGo Search. "
        "Useful for when you need to answer questions about current events. "
        "Input should be a search query."
    )

`name` 变量定义了工具的名称。这正是我们使用 `load_tools` 函数加载工具时所需要的。当然，目前比较棘手的是，`load_tools` 的实现对工具名称做了映射，因此并不是所有工具都如实使用工具类中定义的 `name`。比如，`DuckDuckGoSearchRun` 的名称是 `duckduckgo_search`，但是 `load_tools` 函数需要使用 `ddg-search` 来加载该工具。

请参考源代码 [load_tools.py](https://github.com/langchain-ai/langchain/blob/v0.0.235/langchain/agents/load_tools.py#L314) 了解工具数据初始化的详情。

用法

In [7]:
from langchain.agents import load_tools

tools = load_tools(['ddg-search'])
search = tools[0]
search.run("Who is winner of FIFA worldcup 2018?")

"The 2018 FIFA World Cup was the 21st FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA.It took place in Russia from 14 June to 15 July 2018, after the country was awarded the hosting rights in 2010. It was the eleventh time the championships had been held in Europe, and the first time they were held in Eastern Europe. They reverted back to type to battle past Uruguay and Belgium in the following rounds, but the brilliance of Mbappe, Antoine Griezmann and Paul Pogba shone out in their 4-2 final win over Croatia,... FIFA World Cup Qatar 2022™. World Cup memories of a French superfan. 6 May 2020. 2018 FIFA World Cup Russia™. #WorldCupAtHome: Electric Mbappe helps France win seven-goal thriller. 9 Apr 2020 ... Pele and Brazil won their third title in 1970. Pele, with three trophies, has won the most World Cups by an individual player. Brazil's last World Cup trophy came in 2002. Argentina, powered by the brilliance of the late Diego Maradona,

你应该期望与方法1类似的输出。

最后，分享一个辅助函数 `get_all_tool_names`，用于获取所有工具的名称。

In [8]:
from langchain.agents import get_all_tool_names

get_all_tool_names()

['python_repl',
 'requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-serper',
 'google-serper-results-json',
 'serpapi',
 'twilio',
 'searx-search',
 'wikipedia',
 'arxiv',
 'pupmed',
 'human',
 'awslambda',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'pal-math',
 'pal-colored-objects',
 'llm-math',
 'open-meteo-api']

### Agent

`Agent` 通常需要 `Tool` 配合工作，因此我们将 `Agent` 实例放在 `Tool` 之后。我们以 Zero-shot ReAct 类型的 `Agent` 为例，来看看如何使用。代码如下：

In [10]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["ddg-search", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("What is the height difference between Eiffel Tower and Taiwan 101 Tower?")



> Entering new AgentExecutor chain...


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 7-10: ordinal not in range(256)